Imports

In [23]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from PIL import Image
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import numpy as np

Get Image Size


In [24]:
def get_sample_image_size(dataset_dir):
    """
    dataset_dir: path to something like "Dataset/training"
                 which has subfolders (e.g., glioma_tumor, meningioma_tumor, etc.)
    returns (width, height) of the first found image
    """
    # List subfolders (the class folders)
    class_folders = [f for f in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, f))]
    
    # We assume at least one class folder and at least one image
    first_class_folder = os.path.join(dataset_dir, class_folders[0])
    image_files = [f for f in os.listdir(first_class_folder) 
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    first_image_path = os.path.join(first_class_folder, image_files[0])
    
    # Open the image and check size
    with Image.open(first_image_path) as img:
        width, height = img.size
        print(f"Discovered image size: {width}x{height}")
        return width, height

# Example usage:
dataset_dir = "Dataset/testing"
detected_width, detected_height = get_sample_image_size(dataset_dir)


Discovered image size: 495x619


The Model

In [25]:

########################################
# 1) Define the Model Class
########################################
class LeNet5_512_Dropout(nn.Module):
    def __init__(self, dropout_p=0.5, num_classes=4):
        """
        A LeNet-5-inspired network for 512x512 RGB images, with dropout layers.
        """
        super(LeNet5_512_Dropout, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.pool = nn.AvgPool2d(kernel_size=2)

        # For 512×512 input => after two (conv + pool) => shape is (16×125×125) => 250,000 features
        self.fc1 = nn.Linear(16 * 125 * 125, 120)
        self.drop1 = nn.Dropout(p=dropout_p)
        self.fc2 = nn.Linear(120, 84)
        self.drop2 = nn.Dropout(p=dropout_p)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))   # => (6×508×508)
        x = self.pool(x)           # => (6×254×254)
        
        x = F.relu(self.conv2(x))   # => (16×250×250)
        x = self.pool(x)           # => (16×125×125)
        
        x = x.view(x.size(0), -1)   # flatten => 16*125*125 = 250,000
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = F.relu(self.fc2(x))
        x = self.drop2(x)
        x = self.fc3(x)
        return x


model = LeNet5_512_Dropout(dropout_p=0.5, num_classes=4)

DataLoader

In [26]:
def create_dataloaders():
    """
    Adjust these to match your dataset paths and augmentation strategies.
    """
    # Example normalization stats (like ImageNet)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    

    train_transform = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    val_test_transform = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    # Adjust these to match your folder structure
    train_path = "Dataset/training"
    test_path = "Dataset/testing"

    # Full training dataset
    full_train = datasets.ImageFolder(root=train_path, transform=train_transform)

    # Split into train/val (e.g., 80/20)
    train_size = int(0.8 * len(full_train))
    val_size = len(full_train) - train_size
    train_dataset, val_dataset = random_split(full_train, [train_size, val_size])

    # Override the transform for val
    val_dataset.dataset.transform = val_test_transform

    # Test dataset
    test_dataset = datasets.ImageFolder(root=test_path, transform=val_test_transform)

    # DataLoaders
    batch_size = 8
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader, val_loader, test_loader

Training Function

In [27]:
def train_and_evaluate(
    model, 
    train_loader, 
    val_loader, 
    test_loader,
    num_epochs=5, 
    lr=0.001, 
    weight_decay=1e-5
):
    """
    Trains for `num_epochs`, shows progress bars (via tqdm),
    then returns (trained_model, test_accuracy).
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Optionally use a learning-rate scheduler
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    for epoch in range(num_epochs):
        ################################################
        # TRAIN PHASE
        ################################################
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total_samples = 0

        # Wrap train_loader in tqdm for real-time progress
        train_pbar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] (Train)")
        for images, labels in train_pbar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * images.size(0)
            running_corrects += (preds == labels).sum().item()
            total_samples += labels.size(0)

            current_loss = running_loss / total_samples
            current_acc = running_corrects / total_samples
            train_pbar.set_postfix({
                "loss": f"{current_loss:.4f}",
                "acc": f"{current_acc:.4f}"
            })

        epoch_loss = running_loss / total_samples
        epoch_acc = running_corrects / total_samples

        ################################################
        # VALIDATION PHASE
        ################################################
        model.eval()
        val_loss_accum = 0.0
        val_corrects = 0
        val_samples = 0

        val_pbar = tqdm(val_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] (Val)")
        with torch.no_grad():
            for images, labels in val_pbar:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                val_loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                val_loss_accum += val_loss.item() * images.size(0)
                val_corrects += (preds == labels).sum().item()
                val_samples += labels.size(0)

                current_val_loss = val_loss_accum / val_samples
                current_val_acc = val_corrects / val_samples
                val_pbar.set_postfix({
                    "val_loss": f"{current_val_loss:.4f}",
                    "val_acc": f"{current_val_acc:.4f}"
                })

        val_epoch_loss = val_loss_accum / val_samples
        val_epoch_acc = val_corrects / val_samples

        print(
            f"Epoch {epoch+1}/{num_epochs} => "
            f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f} | "
            f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}"
        )

        scheduler.step()  # reduce LR every `step_size` epochs

    #########################################
    # FINAL TEST EVALUATION
    #########################################
    model.eval()
    test_correct = 0
    test_total = 0

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            test_correct += (preds == labels).sum().item()
            test_total += labels.size(0)

    test_acc = test_correct / test_total
    print(f"Final Test Accuracy = {test_acc:.4f}")
    return model, test_acc

Plots

In [28]:
def plot_metrics(train_losses, train_accuracies, val_losses, val_accuracies):
    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(12, 5))
    
    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label='Train Loss')
    plt.plot(epochs, val_losses, label='Val Loss')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label='Train Acc')
    plt.plot(epochs, val_accuracies, label='Val Acc')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()


Training

In [30]:
if __name__ == "__main__":
    # Build data loaders
    train_loader, val_loader, test_loader = create_dataloaders()

    # Define a small "grid" of hyperparameters
    param_grid = [
        {"dropout_p": 0.3, "lr": 0.001, "weight_decay": 1e-5, "num_epochs": 5},
        {"dropout_p": 0.5, "lr": 0.001, "weight_decay": 1e-5, "num_epochs": 5},
        {"dropout_p": 0.5, "lr": 0.0001, "weight_decay": 1e-4, "num_epochs": 5},
    ]

    best_model = None
    best_acc = 0.0
    best_params = None

    # Iterate over each config
    for i, cfg in enumerate(param_grid):
        print(f"\n==> Starting training for config {i+1}/{len(param_grid)}: {cfg}")

        # 1) Create a new model with config's dropout
        model = LeNet5_512_Dropout(dropout_p=cfg["dropout_p"], num_classes=4)

        # 2) Train & Evaluate on test
        trained_model, test_acc = train_and_evaluate(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            test_loader=test_loader,
            num_epochs=cfg["num_epochs"],
            lr=cfg["lr"],
            weight_decay=cfg["weight_decay"]
        )

        # 3) Track the best
        if test_acc > best_acc:
            best_acc = test_acc
            best_model = trained_model
            best_params = cfg

    # Show the best results
    print("\n====================")
    print(f"Best Test Accuracy: {best_acc:.4f}")
    print(f"Best Params: {best_params}")

    # Save the best model
    if best_model is not None:
        torch.save(best_model.state_dict(), "best_model.pth")
        print("Saved best model to best_model.pth")
    else:
        print("No model was trained or something went wrong.")


==> Starting training for config 1/3: {'dropout_p': 0.3, 'lr': 0.001, 'weight_decay': 1e-05, 'num_epochs': 5}


Epoch [1/5] (Val): 100%|██████████| 72/72 [00:16<00:00,  4.48it/s, val_loss=0.6807, val_acc=0.7160]


Epoch 1/5 => Train Loss: 1.0594, Train Acc: 0.5579 | Val Loss: 0.6807, Val Acc: 0.7160


Epoch [2/5] (Val): 100%|██████████| 72/72 [00:17<00:00,  4.16it/s, val_loss=0.4720, val_acc=0.8118]


Epoch 2/5 => Train Loss: 0.5942, Train Acc: 0.7591 | Val Loss: 0.4720, Val Acc: 0.8118


...

Final Test Accuracy = 0.7081

==> Starting training for config 2/3: {'dropout_p': 0.5, 'lr': 0.001, 'weight_decay': 1e-05, 'num_epochs': 5}


Epoch [1/5] (Val): 100%|██████████| 72/72 [00:19<00:00,  3.66it/s, val_loss=0.8968, val_acc=0.5941]


Epoch 1/5 => Train Loss: 1.2620, Train Acc: 0.4717 | Val Loss: 0.8968, Val Acc: 0.5941


Epoch [2/5] (Train):  90%|████████▉ | 258/287 [01:53<00:12,  2.28it/s, loss=0.8479, acc=0.6487]


KeyboardInterrupt: 

Prediction Functions

In [ ]:

def unnormalize_image(tensor, mean, std):
    """
    Undo the normalization so the image looks 'normal' when plotted.
    tensor: a PyTorch tensor of shape [3, H, W].
    mean, std: lists or tuples of length 3, e.g. mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225].
    Returns a numpy array of shape (H, W, 3) in [0,1] range for plotting.
    """
    # Clone so we don't modify the original tensor
    image = tensor.clone().cpu().numpy()
    
    # For each channel: unnormalize
    for i in range(3):
        image[i] = (image[i] * std[i]) + mean[i]
    
    # Clip to valid range [0,1], just in case
    image = np.clip(image, 0, 1)
    
    # Convert from (C,H,W) => (H,W,C)
    image = np.transpose(image, (1, 2, 0))
    return image

def visualize_random_test_image(model, test_dataset, class_names, device, mean, std):
    """
    1) Picks a random image from test_dataset
    2) Runs the model to get a prediction
    3) Displays the image, true label, and predicted label
    """
    model.eval()
    # Pick a random index
    random_idx = random.randrange(len(test_dataset))
    
    # test_dataset[random_idx] returns (image_tensor, label)
    image_tensor, true_label = test_dataset[random_idx]
    
    # Add batch dimension => shape [1, 3, H, W]
    input_tensor = image_tensor.unsqueeze(0).to(device)

    # Forward pass to get prediction
    with torch.no_grad():
        outputs = model(input_tensor)
        _, predicted_label = torch.max(outputs, 1)
    
    predicted_label = predicted_label.item()
    
    # Convert label indexes to names
    true_class_name = class_names[true_label]
    predicted_class_name = class_names[predicted_label]
    
    # Unnormalize the image for display (optional)
    image_np = unnormalize_image(image_tensor, mean, std)

    # Plot
    plt.figure(figsize=(6, 6))
    plt.imshow(image_np)
    plt.title(f"True: {true_class_name} | Pred: {predicted_class_name}")
    plt.axis("off")
    plt.show()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model.to(device)

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_loader, val_loader, test_dataset = create_dataloaders()
class_names = ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]

Predict

In [ ]:
visualize_random_test_image(best_model, test_dataset, class_names, device, mean, std)